# <center> Готовим данные для файнтюнинга LLM. 🦙 </center> 

<img src='../images/lama_docs.webp' align="right" width="500" height="428" >

В этом ноутбуке подготовим датасет для файнтюнинга LLM и загрузим его на `HuggingFace`. <br>

В качестве источника данных скачаем все посты из Telegram-канала [Datafeeling](https://t.me/datafeeling), через который многие нашли этот курс. В дальнейшем, с помощью этого датасета зафайнтюним модель писать посты в стиле Алерона. <br>

**Чтобы скачать все посты любого публичного TG-канала:**
* в десктоп приложении Telegram переходим к канал
* нажимаем три точки
* выбираем - "Экспорт истории чата"
* там выбираем формат (HTML или json), путь куда сохраняем и какой контент канала хотим скачать
* получаем все посты в одном файле (в нашем случае json)

In [22]:
!pip install datasets langchain langchain-openai openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx-rtd-theme 2.0.0 requires docutils<0.21, but you have docutils 0.21.2 which is incompatible.
mxnet 1.9.1 requires graphviz<0.9.0,>=0.8.1, but you have graphviz 0.20.3 which is incompatible.
bentoml 0.13.2+6.g2dc5913c requires sqlalchemy<1.4.0,>=1.3.0, but you have sqlalchemy 2.0.29 which is incompatible.
anomalib 0.7.0 requires pytorch-lightning<1.10.0,>=1.7.0, but you have pytorch-lightning 2.1.4 which is incompatible.


In [ ]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT с ключом курса и необходимые файлы!
# !mkdir ../data/
# !wget -P ../data https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/data/result.json
# !wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [3]:
import os
from getpass import getpass
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

In [10]:
# указываем путь к файлу, скаченному из телеграма
js_path = '../data/result.json'

# Load the JSON file
with open(js_path, 'r') as file:
    data = json.load(file)

# посмотрим на первое сообщение
data['messages'][2]

{'id': 3,
 'type': 'message',
 'date': '2021-03-21T20:26:01',
 'date_unixtime': '1616343961',
 'edited': '2021-04-11T00:37:28',
 'edited_unixtime': '1618087048',
 'from': '🏆 Data Feeling 🤹',
 'from_id': 'channel1457597091',
 'text': 'Всем привет!\n\nРешил запустить свой канал. Буду рассказывать здесь про свой опыт в Data Science и лайфхаки',
 'text_entities': [{'type': 'plain',
   'text': 'Всем привет!\n\nРешил запустить свой канал. Буду рассказывать здесь про свой опыт в Data Science и лайфхаки'}]}

<div class="alert alert-success">
    
Видим, что помимо текста поста в сообщениях содержится ещё много дополнительной информации. Для наших целей извлечём только тексты.

In [11]:
txt = []
for message in data['messages']:
    if isinstance(message['text'], str):
        txt.append(message['text'])
    else:
        temp_txt = ''
        for m in message['text']:
            if isinstance(m, str):
                temp_txt += m
            else:
                if isinstance(m, dict) and 'text' in m.keys():
                    temp_txt += m['text']
        txt.append(temp_txt)

In [16]:
df = pd.DataFrame({"text": txt})
df.head()

,text
0,
1,
2,Всем привет!\n\nРешил запустить свой канал. Бу...
3,Когда начинаешь погружаться в DS - начинаешь р...
4,#лайфхаки\nПоймал себя на периодическом подгля...


<div class="alert alert-success">
    
Видим, что есть посты без текстов - видео или картинки. <br>
Преобразуем датасет в нужный формат и почистим от пропусков.

In [18]:
df = pd.DataFrame({
    'Instruction': 'Write a post on the following topic', # Инструкция будет везде одинаковая
    'Input': '',                                # поле с темой поста пока оставим пустым
    'Response': [t for t in txt if len(t) != 0] # в Response записываем текст поста и убираем пустые строки
})

df.head()

,Instruction,Input,Response
0,Write a post on the following topic,,Всем привет!\n\nРешил запустить свой канал. Бу...
1,Write a post on the following topic,,Когда начинаешь погружаться в DS - начинаешь р...
2,Write a post on the following topic,,#лайфхаки\nПоймал себя на периодическом подгля...
3,Write a post on the following topic,,"Speed up'ал, speed up'аю и буду speed up'ать с..."
4,Write a post on the following topic,,"За этот месяц раз 5 услышал слово ""экстраполяц..."


In [19]:
# Получилось 587 постов с текстом.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Instruction  587 non-null    object
 1   Input        587 non-null    object
 2   Response     587 non-null    object
dtypes: object(3)
memory usage: 13.9+ KB


## Определяем тематику поста с помощью ChatGPT 🤖

Чтобы самим не заполнять поле input и не читать все 580 сообщений, определяя тему - попросим ЛЛМ сделать это за нас.

In [22]:
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите ваш OpenAI API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш OpenAI API ключ ········


In [35]:
from tqdm import tqdm
tqdm.pandas()

# Функция для определения тематики поста. Примени её ко всему датасету.
def topic_extractor(text):
 
    answer = llm.invoke(f'Extract a phrase that best describes the topic of the following post. \n Post: {text}')
    return str(answer.content)

df['Input'] = df['Response'].progress_apply(topic_extractor)

100%|██████████| 587/587 [07:25<00:00,  1.32it/s]


In [20]:
df.head()

,Instruction,Input,Response
0,Write a post on the following topic,Запуск канала о Data Science,Всем привет!\n\nРешил запустить свой канал. Бу...
1,Write a post on the following topic,Простые примеры кода для новых методов/алгоритмов,Когда начинаешь погружаться в DS - начинаешь р...
2,Write a post on the following topic,Полезные расширения для работы с табличными да...,#лайфхаки\nПоймал себя на периодическом подгля...
3,Write a post on the following topic,Ways to speed up Pytorch training on Kaggle,"Speed up'ал, speed up'аю и буду speed up'ать с..."
4,Write a post on the following topic,Интерпретация моделей в машинном обучении,"За этот месяц раз 5 услышал слово ""экстраполяц..."


<div class="alert alert-success">
    
Видим, что модель отлично справилась с задачей - тематика хорошо определена.

In [36]:
# Сохраняем датасет
df.to_csv('../data/prepared_dataset.csv', index=False)

# Загружаем датасет на HuggingFace 🤗

<div class="alert alert-success">
    
Теперь переходим на [HuggingFace](https://huggingface.co) и создаем новый датасет, затем в него подгружаем получившийся файл. <br>
Теперь датасет доступен всем по ссылке: https://huggingface.co/datasets/Ivanich/datafeeling_posts

In [23]:
# Попробуем его подгрузить прямо с HuggingFace
from datasets import load_dataset

dataset = load_dataset("Ivanich/datafeeling_posts")

Generating train split:   0%|          | 0/587 [00:00<?, ? examples/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Instruction', 'Input', 'Response'],
        num_rows: 587
    })
})

In [26]:
dataset['train'][0]

{'Instruction': 'Write a post on the following topic',
 'Input': 'Запуск канала о Data Science',
 'Response': 'Всем привет!\n\nРешил запустить свой канал. Буду рассказывать здесь про свой опыт в Data Science и лайфхаки'}

### В следующем ноутбуке c помощью этого датасета будем файнтюнить Llama 3.1